
## 1️⃣ What is TF-IDF (quick recap)

For a word **w** in document **d**:

### **TF (Term Frequency)**
$$
[
TF(w, d) = \frac{\text{count of w in d}}{\text{total words in d}}
]
$$
### **IDF (Inverse Document Frequency)**
$$
[
IDF(w) = \log\left(\frac{N}{df(w)}\right)
]
$$
Where:

* ( N ) = total number of documents
* ( df(w) ) = number of documents containing word ( w )

### **TF-IDF**
$$
[
TFIDF(w, d) = TF(w, d) \times IDF(w)
]
$$
---






In [3]:
from collections import Counter, defaultdict
import math

class TFIDFVectorizer:
    """
    A simple TF-IDF Vectorizer implemented from scratch.
    """

    def __init__(self) -> None:
        self.vocabulary_: dict[str, int] = {}
        self.idf_: dict[str, float] = {}
        self.fitted: bool = False

    def _tokenize(self, text: str) -> list[str]:
        return text.lower().split()

    def fit(self, documents: list[str]) -> None:
        """
        Learn vocabulary and IDF values from the documents.
        """
        tokenized_docs: list[list[str]] = [self._tokenize(doc) for doc in documents]
        num_docs: int = len(tokenized_docs)

        # ---------- OPTIMIZED DF COMPUTATION ----------
        df: dict[str, int] = defaultdict(int)

        for doc_tokens in tokenized_docs:
            unique_words = set(doc_tokens)      # O(L)
            for word in unique_words:           # O(U)
                df[word] += 1

        # Build vocabulary
        self.vocabulary_ = {
            word: idx for idx, word in enumerate(df.keys())
        }

        # Compute IDF
        self.idf_ = {
            word: math.log(num_docs / df[word])
            for word in df
        }

        self.fitted = True

    def _compute_tf(self, tokens: list[str]) -> dict[str, float]:
        tf: dict[str, float] = {}
        token_count = Counter(tokens)
        total_tokens = len(tokens)

        for word, count in token_count.items():
            tf[word] = count / total_tokens

        return tf

    def transform(self, documents: list[str]) -> list[list[float]]:
        """
        Transform documents to TF-IDF vectors.
        - Find the tokens.
        - Find the TF for them.
        - generate the list length of whole vocabulary.
        - for each word in the tf. => 
            - Find the id of the word from the vocabulary.
            - fill the list with the tf*idf values.
        """
        if not self.fitted:
            raise ValueError("The vectorizer must be fitted before calling transform().")

        tfidf_vectors: list[list[float]] = []

        for doc in documents:
            tokens = self._tokenize(doc)
            tf = self._compute_tf(tokens)

            vector = [0.0] * len(self.vocabulary_)

            for word, tf_value in tf.items():
                if word in self.vocabulary_:
                    idx = self.vocabulary_[word]
                    vector[idx] = tf_value * self.idf_[word]

            tfidf_vectors.append(vector)

        return tfidf_vectors

    def fit_transform(self, documents: list[str]) -> list[list[float]]:
        """
        Fit to data, then transform it.
        """
        self.fit(documents)
        return self.transform(documents)


In [4]:
documents = [
    "I love machine learning",
    "I love deep learning",
    "Machine learning is fun"
]

vectorizer = TFIDFVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

for vec in tfidf_matrix:
    print(vec)


[0.1013662770270411, 0.0, 0.1013662770270411, 0.1013662770270411, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.1013662770270411, 0.1013662770270411, 0.27465307216702745, 0.0, 0.0]
[0.1013662770270411, 0.0, 0.0, 0.0, 0.0, 0.27465307216702745, 0.27465307216702745]


N - number of documnets
L - avg numbe rof words in the document
V - vocabulary size



| Step / Method          | Time Complexity    | Simple meaning                  |
| ---------------------- | ------------------ | ------------------------------- |
| Tokenization           | O(N × L)           | Read every word once            |
| DF computation         | O(N × L)           | Count unique words per document |
| Vocabulary + IDF       | O(V)               | One pass over unique words      |
| TF computation         | O(N × L)           | Count words per document        |
| TF-IDF vector creation | O(N × (V + L))     | Build vectors of size V         |
| **fit() total**        | **O(N × L)**       | Training is linear              |
| **transform() total**  | **O(N × (V + L))** | Vector creation dominates       |

- space 

| Component       | Space Complexity | Simple meaning            |
| --------------- | ---------------- | ------------------------- |
| Tokenized docs  | O(N × L)         | Store words temporarily   |
| Vocabulary      | O(V)             | One entry per unique word |
| IDF values      | O(V)             | One value per word        |
| TF-IDF matrix   | **O(N × V)**     | Dense document vectors    |
| **Total space** | **O(N × V)**     | Memory-heavy part         |
